In [1]:
import getpass
import os

# Set LangSmith Environment Variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for LangSmith")


In [2]:
# Set up google llm model for RAG usage
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


print("Initializing chat model!")
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")
print("Chat model initialization over!")

Initializing chat model!
Chat model initialization over!


In [3]:
print("Initializing embedder model!")
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
print("Embedder model initialized!")

Initializing embedder model!


/opt/miniconda3/envs/rag-agent-musings/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embedder model initialized!


In [4]:
print("Initialize vector store!")
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
print("Vector store initialized!")

Initialize vector store!
Vector store initialized!


In [5]:
# Loading documents for RAG use

# get beautiful soup
import bs4
# get Document Loaders
from langchain_community.document_loaders import WebBaseLoader
desired_webpage_link = "https://lilianweng.github.io/posts/2023-06-23-agent/"
# the original tutorial only wanted us to keep html tags that
# contain any of the class names listed in the tuple assigned to the 
# class_ keyword, in the below variable. We discard all other html tags.
bs4_filtered = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

webpage_loader = WebBaseLoader(
  web_paths=(desired_webpage_link,),
  bs_kwargs={"parse_only": bs4_filtered}
)

documents = webpage_loader.load()
#for key in documents[0]:
  #print(f"New key: {key}")
#print(f"how many characters are in our page: {len(documents[0].page_content)}")
#print(f"The first two thousand characters of the document: {documents[0].page_content[:2000]}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# create a text splitter instance
recursive_text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=200,
  add_start_index=True,
)

# obtain splitted text. our document is now a bunch of document shards.
splitted_text = recursive_text_splitter.split_documents(documents)
#print(f"The splits we got back:\n\n{splitted_text[4].page_content}")
#print(f"Number of splits:{len(splitted_text)}")

# We've loaded in our document. 
Split it up into text chunks, to allow our model to access bits of the data that fit into the LLM context window.

Now, it's time to put these chunks into a storage format the LLM can access.

In [7]:
# the vector store was previously initialized with our defined embeddings
# in one command sequence, we tell the vector store to embed each of our document chunks.
# we then store the embedded versions of these chunks into our vector store, for 
# later retrieval
vector_store__document_embedded_vectors = vector_store.add_documents(splitted_text)


In [8]:
print(f"vector store shape: {vector_store__document_embedded_vectors}")

vector store shape: ['8ecc7774-2c76-4c9a-bc79-7a6379389f0c', 'a61bd847-6e0f-4ad4-a1ae-9005b292e4bf', '5c911a97-b803-45f9-af64-cd10f792cfb1', 'b067690c-8da9-416c-ae2a-02d3d5af838b', '1c18338e-136a-4b99-9759-7df9e75a108f', 'd9cae80c-2e0d-4c80-8abe-2292ae632c14', '20b56683-1f15-4e1a-819c-32e9eeaec5c1', '24101068-73fd-4ba0-9477-6d275e6d82d7', '427d0666-6b2a-4f16-9e23-78d75ae0378d', '03c3cb3f-34a5-4334-9de0-3b61d5dabcb9', 'ca26fa0e-f1ea-4b22-ba7d-5327892c9394', '4920eb01-bc1d-4110-805b-e238eb5000d0', 'a4f0a59a-025d-468c-8a75-18c64c75ee31', '7ce60e37-8de3-4090-a814-2632c5cdaf66', 'a3b6f0d9-72db-4321-adc2-3ed8d252577a', 'ac2dfa7a-23be-4959-a4bf-ff73bfc7e94e', 'a5bfc19c-785c-4045-b670-4679096ca365', 'd0f2c3d3-4113-4997-ad0f-fb0ef55c128a', 'b7325442-d6ec-426e-8854-15c31d478766', '20970112-9c19-42cf-a467-3359f6fc5713', 'c5e15180-187b-40e2-894e-a7b86c265076', '4ed6e626-71c8-48f0-a987-1248c29ddfe2', '4cf6ac92-bb9d-445c-82a1-d3416d15fa8d', '2e74c9ac-0fa9-462d-b24f-1d0c8a2a421c', '3b6d2af5-cfa6-4177

# We now should be able to answer User Queries
By accessing our vector store and returning the relevant chunks of text, from Lillian's blog post, that answers the user's questions.

## Onto Retrieval And Generation

In [ ]:
# Get RAG prompt
from langchain import hub

# The actual RAG prompt
rag_prompt = hub.pull("rlm/rag-prompt")
#print(f"Actual template response: {rag_prompt.messages[0].prompt.template}.")

example_rag_message = rag_prompt.invoke({
  "context": "context here",
  "question": "question here"
}).to_messages() # Example of passing in context and question to a full RAG prompt.

print(f"Example rag message: {example_rag_message}")

Example rag message: [HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: question here \nContext: context here \nAnswer:", additional_kwargs={}, response_metadata={})]


In [ ]:
## Use LangGraph to generate for our RAG application.

# Begin the state definition phase, where we discuss what type of data we want our RAG model to process.

from langchain_core.documents import Document
from typing_extensions import List, TypedDict

# Create the actual state, to define what data our RAG model handles

class RAG_State(TypedDict):
  question: str
  context: List[Document] # remember, context should be several chunks of a document
  answer: str
